### Sedaro API Propulsion Example

This notebook exercises a basic propulsion configuration in the Starlink demo scenario. The updated scenario will start a 
maneuver procedure immediately on all of the templated satellites.

Each satellite is given a single ion thruster and 20 kg of fuel spread across two separate spherical fuel tanks. The maneuver is statically scheduled based on pre-defined start and end times. Five minutes prior to maneuver start, the satellites will enter a premaneuver pointing mode to ensure proper pointing at maneuver start. After completing the
initial pointing phase, the thruster will be activated with a constant thrust of 500 mN in the positive cross-track direction. The burn continues for one hour. After engine shut-off, regular satellite operations resume.

#### Important: Read Before Running

This notebook makes changes to agent and scenario branches indicated in the settings section. Ensure any changes to the target branches are saved prior to running this code. Sedaro recommends committing current work and creating new branches in the target repositories to avoid loss of work.

This notebook also requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [ ]:
import json
import time
from sedaro import SedaroApiClient
from sedaro_base_client.apis.tags import jobs_api
import matplotlib.pyplot as plt

In [ ]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

# Obtain these IDs from the URL of both the agent template and scenario edit UIs for the starlink 
# vehicle and scenario.
# Example: https://satellite.sedaro.com/#/agent-template-edit/189/targets/targets  -> 189
# Example: https://satellite.sedaro.com/#/scenario/37/edit/time  -> 37
AGENT_TEMPLATE_BRANCH_ID = ...  # ID of starlink vehicle
SCENARIO_BRANCH_ID = ...        # ID of starlink scenario
HOST = 'https://api.sedaro.com'

In [ ]:
def get_by_name(block_class, name):
    '''Temporary workaround for searching by a particular field.'''
    instances = block_class.get_all()
    results = [entry for entry in instances if entry.name == name]
    if len(results) > 0:
        return results[0]
    else:
        return None

#### Build Necessary Components

Here each new component is instantiated using the `branch_client.<BLOCK_NAME>.create` method. See the `sedaro` package documentation [here](https://github.com/sedaro/sedaro-python/tree/main/sedaro) for a listing of available blocks. We integrate the new components into the existing elements of the demo scenario by querying information with the `.get_all` and `.get_first` block methods.

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    # Instantiate objects we will need
    branch_client = sedaro_client.get_branch(AGENT_TEMPLATE_BRANCH_ID)
    conops = branch_client.ConOps.get_first()


    # Create fuel reservoir
    reservoir_name = "Satellite Fuel Reservoir"
    fuel_reservoir = get_by_name(branch_client.FuelReservoir, reservoir_name)
    if fuel_reservoir is None:
        fuel_reservoir = branch_client.FuelReservoir.create(
            name=reservoir_name,
            flowRate=0.0,
        )


    # Create two spherical fuel tanks
    spherical_tank_locations = [
        (-1., -1., 0.),
        (1., 1., 0.),
    ]
    subsystem = get_by_name(branch_client.Subsystem, 'GNC')
    spherical_tanks = []
    for idx, location in enumerate(spherical_tank_locations):
        tank_name = f"Spherical Fuel Tank {idx}"
        spherical_tank = get_by_name(branch_client.SphericalFuelTank, tank_name)
        if spherical_tank is None:
            spherical_tank = branch_client.SphericalFuelTank.create(
                name=tank_name,
                componentType="SPHERICAL_FUEL_TANK",
                capacity=10.,
                wetMass=10.,
                priority=1,
                location=location,
                reservoir=fuel_reservoir.id,
                inertia=[[0.75, 0., 0.], [0., 0.75, 0.], [0., 0., 0.75]],
                diameter=0.25,
                subsystem=subsystem.id,
            )
        spherical_tanks.append(spherical_tank)


    # Create body frame vector that will be used to orient the thruster
    bfv_name = "Tank BFV"
    bfv = get_by_name(branch_client.BodyFrameVector, bfv_name)
    if bfv is None:
        bfv = branch_client.BodyFrameVector.create(
            name=bfv_name,
            definitionType="VECTOR",
            definitionParams={
                'vector': [-1., 0., 0.],
            },
        )


    # Create thruster
    power_bus = get_by_name(branch_client.BusRegulator, "12V Reg")
    thruster_name = "Main Engine"
    thruster = get_by_name(branch_client.Thruster, thruster_name)
    if thruster is None:
        thruster = branch_client.Thruster.create(
            name=thruster_name,
            componentType='THRUSTER',
            subsystem=subsystem.id,
            busRegulator=power_bus.id,
            isp=1000,
            minThrust=0.1,
            maxThrust=0.5,
            location=[-0.5, 0, 0],
            orientation=bfv.id,
            fuelReservoir=fuel_reservoir.id
        )


    # Create thrust control algorithm
    algorithm_name = "Static Thrust Algorithm"
    algorithm = get_by_name(
        branch_client.StaticThrustControlAlgorithm, algorithm_name
    )
    if algorithm is None:
        algorithm = branch_client.StaticThrustControlAlgorithm.create(
            name=algorithm_name,
            algorithmType='THRUST_CONTROL',
            algorithmSubtype='STATIC',
            rate=1,
            thrusters={
                thruster.id: {'thrust': 0.5},
            },
        )


    # Pointing mode
    #
    # Going to duplicate an existing pointing mode here that will point our thruster in
    # the anti-ram direction.
    ref_pointing_mode = get_by_name(branch_client.MaxAlignPointingMode, 'Nadir Point')
    pointing_mode_name = "Thrust Pointing Mode"
    premaneuver_pointing_mode = ref_pointing_mode
    maneuver_pointing_mode = get_by_name(branch_client.MaxAlignPointingMode, pointing_mode_name)
    if maneuver_pointing_mode is None:
        maneuver_pointing_mode = branch_client.MaxAlignPointingMode.create(
            name=pointing_mode_name,
            conOps=conops.id,
            pointingModeType='MAX_SECONDARY_ALIGN',
            lockVector=ref_pointing_mode.lockVector.id,
            lockBodyFrameVector=ref_pointing_mode.lockBodyFrameVector.id,
            maxAlignVector=ref_pointing_mode.maxAlignVector.id,
            maxAlignBodyFrameVector=ref_pointing_mode.maxAlignBodyFrameVector.id,
            acAlgorithm=ref_pointing_mode.acAlgorithm.id,
            adAlgorithm=ref_pointing_mode.adAlgorithm.id,
            odAlgorithm=ref_pointing_mode.odAlgorithm.id,
            tcAlgorithm=algorithm.id,
        )


    # Conditions
    premaneuver_start = 59898.270833333328482
    maneuver_start = premaneuver_start + 5 * 60 / 86400
    maneuver_end = maneuver_start + 60 * 60 / 86400

    precondition_name = "Premaneuver Condition"
    precondition = get_by_name(branch_client.Condition, precondition_name)
    if precondition is None:
        precondition = branch_client.Condition.create(
            name=precondition_name,
            conOps=conops.id,
            paramACategory='TIME',
            paramBCategory='SCALAR',
            relationship='LESS',
            scalar=premaneuver_start,
        )

    start_condition_name = "Maneuver Start Condition"
    start_condition = get_by_name(branch_client.Condition, start_condition_name)
    if start_condition is None:
        start_condition = branch_client.Condition.create(
            name=start_condition_name,
            conOps=conops.id,
            paramACategory='TIME',
            paramBCategory='SCALAR',
            relationship='GREATER',
            scalar=maneuver_start,
        )

    end_condition_name = "Maneuver End Condition"
    end_condition = get_by_name(branch_client.Condition, end_condition_name)
    if end_condition is None:
        end_condition = branch_client.Condition.create(
            name=end_condition_name,
            conOps=conops.id,
            paramACategory='TIME',
            paramBCategory='SCALAR',
            relationship='LESS',
            scalar=maneuver_end,
        )


    # Operational Modes
    opmode_name = "Premaneuver Mode"
    opmode = get_by_name(branch_client.OperationalMode, opmode_name)
    if opmode is None:
        opmode = branch_client.OperationalMode.create(
            name=opmode_name,
            conOps=conops.id,
            priority=10000,  # Override everything
            pointingMode=premaneuver_pointing_mode.id,
            conditions=[precondition.id],
        )

    opmode_name = "Maneuver Mode"
    opmode = get_by_name(branch_client.OperationalMode, opmode_name)
    if opmode is None:
        opmode = branch_client.OperationalMode.create(
            name=opmode_name,
            conOps=conops.id,
            priority=10000,  # Override everything
            pointingMode=maneuver_pointing_mode.id,
            conditions=[start_condition.id, end_condition.id],
        )


#### Simulate & Plot Results

Now we will use the API to start the simulation and plot some of the results. The results of this run will also appear in the web interface as usual.

In [ ]:
def poll(api_instance, branch_id, sleep=2):
    """Polls a job until it is complete"""
    response = api_instance.get_simulations(path_params={'branchId': branch_id}, query_params={'latest': ''})
    start = time.time()

    while response.body[0]['status'] == 'RUNNING':
        progress = response.body[0]['progress']['percentComplete']
        t = time.time() - start
        print(f"Progress: {str(progress)[:5]}%, Time: {t:.2f}s".ljust(80), end='\r')
        time.sleep(sleep)
        response = api_instance.get_simulations(path_params={'branchId': branch_id}, query_params={'latest': ''})

    time.sleep(5)

    return response.body[0]

In [ ]:
with SedaroApiClient(api_key=API_KEY, host=HOST) as sedaro_client:
    # Instantiate API and scenario objects
    api_instance = jobs_api.JobsApi(sedaro_client)
    scenario = sedaro_client.get_branch(SCENARIO_BRANCH_ID)

    # Start scenario and wait for it to finish, then get data
    api_instance.start_simulation(path_params={'branchId': scenario.id})
    job = poll(api_instance, scenario.id)
    raw_data = sedaro_client.get_data(job.dataId)

In [ ]:
# Plot results with Matplotlib
times, thrust = [], []
series = raw_data['Data']['series']
for streamId, (time, data) in series.items():
    agent_id, engine_id = streamId.split('/')

    # Extract GNC engine data
    if engine_id == '1':

        # Plot thrust
        times = [(entry - time[0]) * 86400 for entry in time]
        thrust = data[agent_id][thruster.id]['thrust']
        plt.plot(times, thrust)
        plt.ylabel('Thrust (N)')
        plt.xlabel('Elapsed Time (s)')
        plt.show()

        # Plot tank masses
        for tank in spherical_tanks:
            mass = data[agent_id][tank.id]['wetMass']
            plt.plot(times, mass)
        plt.legend([tank.name for tank in spherical_tanks])
        plt.xlabel('Time (s)')
        plt.ylabel('Mass (kg)')
        plt.show()

        # Plot semimajor axis
        sma = data[agent_id]['orbitalElements']['a']
        plt.plot(times, sma)
        plt.xlabel('Time (s)')
        plt.ylabel('Semimajor Axis (km)')
        plt.show()

        break